In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Output, Input
import dash_ag_grid as dag
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
import finlab
from finlab import data
from query.local import Queryer
import plotly.graph_objects as go
from visual.utils import card_generate, candlestick_plot

In [2]:
# Configuration
conn_params = {
"host" : "localhost",
"database" : "Fin_proj",
"user" : "postgres",
"password" : "nckumark"
}
finlab.login("imYEE5zw6nTjQ2NxzJ7ToVfEOYW67YeK4qIigkessdnwkqZuar4dK1e7qYanqoNw#vip_m")


輸入成功!


In [3]:
#*** Change these two parts
# Plug-in here Strategy
    # Data must include open and close for wr_df
close = data.get("price:收盤價")
open = data.get("price:開盤價")
vol = data.get("price:成交股數")
vol_ma = vol.average(10)
rev = data.get('monthly_revenue:當月營收')
rev_year_growth = data.get('monthly_revenue:去年同月增減(%)')
rev_month_growth = 	data.get('monthly_revenue:上月比較增減(%)')
# Strategy start here
#===================================================================================
# 股價創年新高
cond1 = (close == close.rolling(250).max())

# 排除月營收連3月衰退10%以上
cond2 = ~(rev_year_growth < -10).sustain(3) 

# 排除月營收成長趨勢過老(12個月內有至少8個月單月營收年增率大於60%)
cond3 = ~(rev_year_growth > 60).sustain(12,8)

# 確認營收底部，近月營收脫離近年谷底(連續3月的"單月營收近12月最小值/近月營收" < 0.8)
cond4 = ((rev.rolling(12).min())/(rev) < 0.8).sustain(3)

# 單月營收月增率連續3月大於-40%
cond5 = (rev_month_growth > -40).sustain(3)

# 流動性條件
cond6 = vol_ma > 200*1000

buy = cond1 & cond2  & cond3 & cond4 & cond5 & cond6

# 買比較冷門的股票
buy = vol_ma*buy
buy = buy[buy>0]
final_cond = buy.is_smallest(5)
#===================================================================================
if in_period:
    select_cond = final_cond[final_cond.index==start_date]
    # Create selection df
    select_series = select_cond.apply(lambda x: set(select_cond.columns[x]), axis =1)
    select_df = (select_series
                .explode()
                .reset_index()
                .rename(columns={0: "stock_id"}))
    select_df['strategy_name'] = strategy_name
    select_df['select_status'] = True
else:
    select_df = None
# Configuration
conn_params = {
"host" : "localhost",
"database" : "Fin_proj",
"user" : "postgres",
"password" : "nckumark"
}
finlab.login("imYEE5zw6nTjQ2NxzJ7ToVfEOYW67YeK4qIigkessdnwkqZuar4dK1e7qYanqoNw#vip_m")

today_date = pd.Timestamp.today().normalize()
wr_base_date = pd.Timestamp(year=today_date.year, month=1, day=1) # 01/01/year
queryer = Queryer(today_date, conn_params)
start_date = queryer.start_date
start_date_str = start_date.strftime('%Y/%m/%d')
updating_list = updating_period()
strategy_name = '藏敖'
in_period = start_date in updating_list
select_df, wr_df = strategy(strategy_name, start_date, wr_base_date, in_period)

輸入成功!


In [4]:
select_df